## Lệnh cho ngói và gạch

In [ ]:
import json
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# --- CẤU HÌNH ---
JSON_FILENAME = "data_viglacera.json"
CATEGORY_URL = { # Key: địa chỉ loại sản phẩm, value: giá trị chỗ phần selector css tuong ung
    "https://viglaceratiles.vn/san-pham/ngoi-lop.html": ".product-box-tiles",
    "https://viglaceratiles.vn/san-pham/gach-op-lat.html": ".product-box",
}

# Headers giả lập trình duyệt cho requests
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7'
}

def setup_driver():
    """Thiết lập và khởi tạo Selenium Driver"""
    chrome_options = Options()
    # chrome_options.add_argument("--headless") # Bỏ comment nếu muốn chạy ẩn
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def get_all_product_links(url):
    """
    Bước 1: Sử dụng Selenium để cuộn trang và lấy toàn bộ LINK sản phẩm.
    Trả về: Danh sách (List) các đường dẫn URL.
    """
    driver = setup_driver()
    product_links = []

    try:
        print(f"🚀 BƯỚC 1: Khởi động Selenium để lấy link từ: {url}")
        driver.get(url)
        time.sleep(2)

        # --- Logic cuộn trang (Lazy Load) ---
        print("🔄 Đang cuộn trang...")
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Cuộn xuống đáy
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) # Chờ load

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("-> Đã cuộn đến cuối trang.")
                break
            last_height = new_height
            print("... Đang tải thêm ...")

        # --- Phân tích HTML để lấy Link ---
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_items = soup.select('.product-box-tiles')

        print(f"✅ Tìm thấy {len(product_items)} thẻ sản phẩm.")

        for item in product_items:
            link_tag = item.select_one('a.link-load')
            if link_tag and link_tag.get('href'):
                full_link = link_tag.get('href')
                if not full_link.startswith('http'):
                    full_link = "https://viglaceratiles.vn" + full_link
                product_links.append(full_link)

    except Exception as e:
        print(f"❌ Lỗi Selenium: {e}")
    finally:
        driver.quit()

    # Loại bỏ link trùng lặp (nếu có)
    return list(set(product_links))



def scrape_product_detail(url):
    """
    Truy cập URL chi tiết, tải HTML và trích xuất thông tin sản phẩm Viglacera.
    """
    # print(f"🔄 Đang xử lý: {url}") # Bỏ comment nếu muốn xem log chi tiết

    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200: return None
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. Lấy thông tin CỐ ĐỊNH (Mọi sản phẩm đều phải có)
        product_code = soup.select_one('.title-main h2 strong').text.strip() if soup.select_one('.title-main h2 strong') else "N/A"

        breadcrumb = soup.select_one('.breadcrumb li:last-child a')
        collection = breadcrumb.text.strip() if breadcrumb else "N/A"

        images = []
        for img in soup.select('.detail-pic img'):
            if img.get('src'):
                src = "https://viglaceratiles.vn" + img.get('src') if not img.get('src').startswith('http') else img.get('src')
                images.append(src)

        # 2. Lấy thông tin ĐỘNG (Duyệt qua des-item)
        # Bất kể web có "Xương", "Chủng loại", "Công năng" hay "Độ dày"... code đều lấy hết.
        dynamic_specs = {}
        spec_items = soup.select('.des-item')

        for item in spec_items:
            key_tag = item.select_one('span')
            val_tag = item.select_one('h3')

            if key_tag and val_tag:
                # Lưu nguyên văn tên thuộc tính (Ví dụ: "Kích thước", "Xương", "Chủng loại")
                # Dùng .title() để viết hoa chữ cái đầu cho đẹp (Kích Thước)
                key = key_tag.text.strip()
                value = val_tag.text.strip()
                dynamic_specs[key] = value

        # 3. GỘP DỮ LIỆU (Merge)
        # Tạo dictionary chứa thông tin cơ bản trước
        final_data = {
            'URL': url,
            'Mã Sản Phẩm': product_code,
            'Bộ Sưu Tập': collection,
            'Hình Ảnh': images[0] if images else "N/A",
            'Danh Sách Ảnh': images
        }

        # --- ĐÂY LÀ PHẦN QUAN TRỌNG NHẤT ---
        # Gộp toàn bộ thông số động vào kết quả cuối cùng
        # Nếu sản phẩm có "Độ dày", nó sẽ tự thêm key "Độ dày" vào final_data
        final_data.update(dynamic_specs)

        return final_data

    except Exception as e:
        print(f"Lỗi: {e}")
        return None

def main():
    # 1. Lấy danh sách Link
    all_links = get_all_product_links(CATEGORY_URL)

    if not all_links:
        print("Không tìm thấy link nào. Kết thúc.")
        return

    print(f"\n🚀 BƯỚC 2: Bắt đầu cào chi tiết {len(all_links)} sản phẩm bằng Requests...")

    final_data = []

    # 2. Vòng lặp cào chi tiết
    for index, link in enumerate(all_links):
        print(f"[{index + 1}/{len(all_links)}] Xử lý: {link}")

        detail = scrape_product_detail(link)
        if detail:
            final_data.append(detail)

        # Ngủ nhẹ 1 chút để tránh spam server quá nhanh
        # time.sleep(0.5)

    # 3. Lưu ra file JSON
    print(f"\n💾 Đang lưu {len(final_data)} sản phẩm vào file JSON...")

    with open(JSON_FILENAME, 'w', encoding='utf-8') as f:
        # ensure_ascii=False để hiển thị tiếng Việt đúng
        # indent=4 để format đẹp, dễ đọc
        json.dump(final_data, f, ensure_ascii=False, indent=4)

    print(f"🎉 HOÀN TẤT! File lưu tại: {JSON_FILENAME}")

if __name__ == "__main__":
    main()

## ỐNG KHÍ ??? AAC

In [22]:
import json
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# --- CẤU HÌNH ---
JSON_FILENAME = "data_viglacera.json"
CATEGORY_URL = "https://viglacera-aac.vn/collections/tat-ca-san-pham"

# Headers giả lập trình duyệt cho requests
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7'
}

def setup_driver():
    """Thiết lập và khởi tạo Selenium Driver"""
    chrome_options = Options()
    # chrome_options.add_argument("--headless") # Bỏ comment nếu muốn chạy ẩn
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def get_all_product_links(url):
    """
    Bước 1: Sử dụng Selenium để cuộn trang và lấy toàn bộ LINK sản phẩm.
    Trả về: Danh sách (List) các đường dẫn URL.
    """
    driver = setup_driver()
    product_links = []

    try:
        print(f"🚀 BƯỚC 1: Khởi động Selenium để lấy link từ: {url}")
        driver.get(url)
        time.sleep(2)

        # --- Logic cuộn trang (Lazy Load) ---
        print("🔄 Đang cuộn trang...")
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Cuộn xuống đáy
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) # Chờ load

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("-> Đã cuộn đến cuối trang.")
                break
            last_height = new_height
            print("... Đang tải thêm ...")

        # --- Phân tích HTML để lấy Link ---
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_items = soup.select('.product-title')

        print(f"✅ Tìm thấy {len(product_items)} thẻ sản phẩm.")

        for item in product_items:
            link_tag = item.select_one('.product-title a')
            if link_tag and link_tag.get('href'):
                full_link = link_tag.get('href')
                if not full_link.startswith('http'):
                    full_link = url + full_link
                product_links.append(full_link)

    except Exception as e:
        print(f"❌ Lỗi Selenium: {e}")
    finally:
        driver.quit()

    # Loại bỏ link trùng lặp (nếu có)
    return list(set(product_links))



def scrape_product_detail(url):
    print(f"🔄 Đang xử lý: {url}")
    try:
        # 1. Gửi Request
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # --- 2. LẤY THÔNG TIN CƠ BẢN ---

        # Tên sản phẩm
        name_tag = soup.find('h1', itemprop='name')
        product_name = name_tag.text.strip() if name_tag else "N/A"

        # Thương hiệu
        brand_tag = soup.select_one('.pro-brand a')
        brand = brand_tag.text.strip() if brand_tag else "N/A"

        # Loại sản phẩm
        type_tag = soup.select_one('.pro-type a')
        product_type = type_tag.text.strip() if type_tag else "N/A"

        # --- 3. LẤY HÌNH ẢNH ---
        images = []
        # Ảnh chính (Lớn)
        main_img = soup.select_one('#ProductPhoto img')
        if main_img and main_img.get('src'):
            src = main_img.get('src')
            if src.startswith('//'): src = 'https:' + src
            images.append(src)

        # Ảnh phụ (Slider/Thumbnails)
        thumb_imgs = soup.select('#sliderproduct img') # Hoặc selector của slider
        for img in thumb_imgs:
            src = img.get('src')
            if src:
                if src.startswith('//'): src = 'https:' + src
                if src not in images:
                    images.append(src)

        # --- 4. XỬ LÝ BẢNG THÔNG SỐ (TỰ ĐỘNG HÓA SỐ CỘT) ---
        specs = {}
        table = soup.find('table')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                # Lấy tất cả text trong các ô (td hoặc th)
                cols = row.find_all(['td', 'th'])
                row_data = [c.text.strip() for c in cols if c.text.strip()]

                # Bỏ qua dòng rỗng hoặc dòng tiêu đề bảng (chứa chữ "Chỉ tiêu", "Thông số")
                if not row_data: continue
                first_cell_lower = row_data[0].lower()
                if 'chỉ tiêu' in first_cell_lower or 'thông số' in first_cell_lower or 'đơn vị' in first_cell_lower:
                    continue

                # LOGIC DYNAMIC:
                # Cột đầu = Key (Tên)
                # Cột cuối = Value (Giá trị)
                # Các cột giữa = Đơn vị tính (Gộp vào Key)
                if len(row_data) >= 2:
                    key_name = row_data[0]
                    value = row_data[-1]

                    if len(row_data) > 2:
                        # Có cột giữa -> Gộp vào tên
                        units = " ".join(row_data[1:-1])
                        full_key = f"{key_name} ({units})"
                    else:
                        # Chỉ có 2 cột -> Tên thuần túy
                        full_key = key_name

                    specs[full_key] = value

        # --- 5. XỬ LÝ THÔNG TIN MÔ TẢ (LIST UL/LI) ---
        info_dict = {}

        # Tìm thẻ UL chứa thông tin (Thường nằm sau H2 "Thông tin sản phẩm")
        headers = soup.find_all('h2')
        target_ul = None
        for h2 in headers:
            if "THÔNG TIN" in h2.text.upper() or "TÍNH NĂNG" in h2.text.upper():
                # Tìm thẻ UL hoặc DIV chứa UL ngay sau tiêu đề
                sibling = h2.find_next_sibling(['ul', 'div'])
                if sibling:
                    if sibling.name == 'div':
                        target_ul = sibling.find('ul')
                    else:
                        target_ul = sibling
                    if target_ul: break # Tìm thấy thì dừng

        # Nếu không tìm thấy theo H2, tìm UL đầu tiên trong nội dung
        if not target_ul:
            content_div = soup.find('div', class_='pro-tabcontent')
            if content_div:
                target_ul = content_div.find('ul')

        # Duyệt qua các dòng li để lấy dữ liệu
        if target_ul:
            lis = target_ul.find_all('li')
            for i, li in enumerate(lis):
                text = li.text.strip()
                if ':' in text:
                    # Tách theo dấu hai chấm đầu tiên
                    parts = text.split(':', 1)
                    k = parts[0].strip()
                    v = parts[1].strip()
                    info_dict[k] = v
                else:
                    # Dòng không có key:value -> Đặt key tự động
                    info_dict[f"Info_{i+1}"] = text

        # --- 6. TỔNG HỢP KẾT QUẢ ---
        final_data = {
            'URL': url,
            'Tên Sản Phẩm': product_name,
            'Thương Hiệu': brand,
            'Loại Sản Phẩm': product_type,
            'Ảnh Đại Diện': images[0] if images else "N/A",
            'Danh Sách Ảnh': images
        }

        # Gộp thông tin mô tả
        final_data.update(info_dict)
        # Gộp thông số kỹ thuật
        final_data.update(specs)

        return final_data

    except Exception as e:
        print(f"⚠️ Lỗi ngoại lệ: {e}")
        return None

def main():
    # 1. Lấy danh sách Link
    all_links = get_all_product_links(CATEGORY_URL)

    if not all_links:
        print("Không tìm thấy link nào. Kết thúc.")
        return

    print(f"\n🚀 BƯỚC 2: Bắt đầu cào chi tiết {len(all_links)} sản phẩm bằng Requests...")

    final_data = []

    # 2. Vòng lặp cào chi tiết
    for index, link in enumerate(all_links):
        print(f"[{index + 1}/{len(all_links)}] Xử lý: {link}")

        detail = scrape_product_detail(link)
        if detail:
            final_data.append(detail)

        # Ngủ nhẹ 1 chút để tránh spam server quá nhanh
        # time.sleep(0.5)

    # 3. Lưu ra file JSON
    print(f"\n💾 Đang lưu {len(final_data)} sản phẩm vào file JSON...")

    with open(JSON_FILENAME, 'w', encoding='utf-8') as f:
        # ensure_ascii=False để hiển thị tiếng Việt đúng
        # indent=4 để format đẹp, dễ đọc
        json.dump(final_data, f, ensure_ascii=False, indent=4)

    print(f"🎉 HOÀN TẤT! File lưu tại: {JSON_FILENAME}")

if __name__ == "__main__":
    main()

🚀 BƯỚC 1: Khởi động Selenium để lấy link từ: https://viglacera-aac.vn/collections/tat-ca-san-pham
🔄 Đang cuộn trang...
-> Đã cuộn đến cuối trang.
✅ Tìm thấy 6 thẻ sản phẩm.

🚀 BƯỚC 2: Bắt đầu cào chi tiết 6 sản phẩm bằng Requests...
[1/6] Xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-10
🔄 Đang xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-10
[2/6] Xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-12
🔄 Đang xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-12
[3/6] Xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-7
🔄 Đang xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-7
[4/6] Xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-11
🔄 Đang xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/products/san-pham-11
[5/6] Xử lý: https://viglacera-aac.vn/collections/tat-ca-san-pham/pro